# Clone repo

In [1]:
!git clone https://github.com/oleja1shpep/ASR.git

Cloning into 'ASR'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 374 (delta 203), reused 276 (delta 105), pack-reused 0 (from 0)
Receiving objects: 100% (374/374), 79.80 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (203/203), done.


# Install requirements

In [2]:
!pip install -r ASR/requirements.txt -q

     / 553.6 kB 1.9 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Download checkpoints

In [3]:
!gdown https://drive.google.com/uc?id=1LVKpi6Nu7Rk8FcNs88paNXbCqe52kl7e -O model_best.pth

Downloading...
From (original): https://drive.google.com/uc?id=1LVKpi6Nu7Rk8FcNs88paNXbCqe52kl7e
From (redirected): https://drive.google.com/uc?id=1LVKpi6Nu7Rk8FcNs88paNXbCqe52kl7e&confirm=t&uuid=e365b112-01a8-4395-be1d-580f160a5b56
To: /content/model_best.pth
100% 221M/221M [00:04<00:00, 48.3MB/s]


# Run inference on src datasets

Basically the commands looks like:

```
!python ASR/inference.py inferencer.from_pretrained="./model_best.pth" \
inferencer.save_preds_path="predictions"  \
datasets.dataset._target_="src.datasets.<name>" \
datasets.dataset.part="<part>" datasets.dataset.arg1="..."
```

The predictions will be saved to predictions/dataset folder

If you want to save targets to the directory just add inferencer.save_targets_path arguement to command

Example:

In [4]:
# preds will be stored in predictions/dataset folder, targets will be stored in targets/dataset folder

!python ASR/inference.py inferencer.from_pretrained="./model_best.pth" \
inferencer.save_preds_path="predictions" \
inferencer.save_targets_path="targets" \
datasets.dataset._target_="src.datasets.LibrispeechDataset" \
datasets.dataset.part="test-clean"

[2025-10-16 19:51:15,603][torchaudio.utils.download][INFO] - Downloading decoder-assets/librispeech-3-gram/lexicon.txt to /root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lexicon.txt
100% 4.97M/4.97M [00:00<00:00, 91.6MB/s]
[2025-10-16 19:51:15,734][torchaudio.utils.download][INFO] - Downloading decoder-assets/librispeech-3-gram/tokens.txt to /root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/tokens.txt
100% 57.0/57.0 [00:00<00:00, 179kB/s]
[2025-10-16 19:51:15,776][torchaudio.utils.download][INFO] - Downloading decoder-assets/librispeech-3-gram/lm.bin to /root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lm.bin
100% 1.69G/1.69G [00:08<00:00, 206MB/s]
Loading part test-clean
Preparing librispeech folders: test-clean: 100% 87/87 [00:08<00:00, 10.02it/s]
Conformer(
  (conv_subsampling): Conv1dSubsampling(
    (layers): Sequential(
      (0): Conv1d(128, 128, kernel_size=(5,), stride=(3,), padding=(1,))
      (1): ReLU()
    )
  )
  (li

# Calc metrics on src datasets

to calc metrics you need targets directory path, so you'd better add inferencer.save_targets_path arguement to the script above

In [5]:
!python ASR/calc_metrics.py target_dir="targets/dataset" predictions_dir="predictions/dataset"

[2025-10-16 19:54:27,317][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lexicon.txt) exists. Skipping the download.
[2025-10-16 19:54:27,318][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/tokens.txt) exists. Skipping the download.
[2025-10-16 19:54:27,318][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lm.bin) exists. Skipping the download.
    CER_(Beam_Search): 0.06156893533399329
    WER_(Beam_Search): 0.13798817778374514


# Run inference on custom dataset

In [6]:
# download some dataset

!gdown https://drive.google.com/uc?id=1s2f_IhxJUV7RxmExwx_rCvvPSSG81nMi
!unzip sanity_test_data.zip

Downloading...
From: https://drive.google.com/uc?id=1s2f_IhxJUV7RxmExwx_rCvvPSSG81nMi
To: /content/sanity_test_data.zip
100% 747k/747k [00:00<00:00, 7.92MB/s]
Archive:  sanity_test_data.zip
   creating: test_data/
   creating: test_data/audio/
  inflating: test_data/audio/84-121550-0000.flac  
  inflating: test_data/audio/84-121550-0001.flac  
  inflating: test_data/audio/84-121550-0002.flac  
  inflating: test_data/audio/84-121550-0003.flac  
  inflating: test_data/audio/84-121550-0004.flac  
   creating: test_data/transcriptions/
  inflating: test_data/transcriptions/84-121550-0000.txt  
  inflating: test_data/transcriptions/84-121550-0001.txt  
  inflating: test_data/transcriptions/84-121550-0002.txt  
  inflating: test_data/transcriptions/84-121550-0003.txt  
  inflating: test_data/transcriptions/84-121550-0004.txt  


In [7]:
# simulate the transcriptions absence
# !rm -rf ./test_data/transcriptions

Basically the command template looks like this:


```
!python ASR/inference.py -cn=inference_custom \
inferencer.from_pretrained="./model_best.pth" \
datasets.custom.data_dir="<path_to_dataset>" \
inferencer.save_preds_path="predictions" \
inferencer.save_targets_path="targets" \ # optional
```

The argument inferencer.save_targets_path is optional cause you already have your transcriptions in folder \<path_to_dataset\>/transcriptions

In [8]:
# preds will be stored in predictions/custom folder, targets will be stored in targets/custom folder

!python ASR/inference.py -cn=inference_custom inferencer.from_pretrained="./model_best.pth" \
datasets.custom.data_dir="test_data" \
inferencer.save_preds_path="predictions" \
inferencer.save_targets_path="targets"

[2025-10-16 19:54:47,988][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lexicon.txt) exists. Skipping the download.
[2025-10-16 19:54:47,989][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/tokens.txt) exists. Skipping the download.
[2025-10-16 19:54:47,989][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lm.bin) exists. Skipping the download.
Conformer(
  (conv_subsampling): Conv1dSubsampling(
    (layers): Sequential(
      (0): Conv1d(128, 128, kernel_size=(5,), stride=(3,), padding=(1,))
      (1): ReLU()
    )
  )
  (linear): Linear(in_features=128, out_features=256, bias=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (conformer_blocks): ModuleList(
    (0-11): 12 x ConformerBlock(
      (ffn1): FeedForward(
        (layers): Sequential(
          (0): LayerNorm((2

# Calculate metrics on custom data

In [9]:
# do not forget to add 'custom' to the end of predictions_dir
!python ASR/calc_metrics.py target_dir="targets/custom" predictions_dir="predictions/custom"

[2025-10-16 19:55:00,258][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lexicon.txt) exists. Skipping the download.
[2025-10-16 19:55:00,258][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/tokens.txt) exists. Skipping the download.
[2025-10-16 19:55:00,258][torchaudio.utils.download][INFO] - The local file (/root/.cache/torch/hub/torchaudio/decoder-assets/librispeech-3-gram/lm.bin) exists. Skipping the download.
    CER_(Beam_Search): 0.025568753142282553
    WER_(Beam_Search): 0.054296066252587996
